In [1]:
import requests # to get data from link
from bs4 import BeautifulSoup # to scrap data

In [2]:
url="https://www.goodreads.com/quotes/tag/{}?page={}"

In [3]:
emotions = ['love', 'religion', 'philosophy', 'happiness']

In [4]:
complete = url.format(emotions[0],1)

In [5]:
complete

'https://www.goodreads.com/quotes/tag/love?page=1'

In [6]:
def get_quotes(complete):
    '''
    scraping data and storing it in the form array
    '''
    data = requests.get(complete)
    soup = BeautifulSoup(data.text)
    divs = soup.find_all('div', attrs={'class' : 'quoteText'})
    quotes = [div.text.strip().split('\n')[0][1:-1] for div in divs]
    return quotes

In [7]:
quotes = get_quotes(complete)

In [8]:
quotes[12]

'Love all, trust a few, do wrong to none.'

In [9]:
X,y = [], []
# storing scraped data in ythe form of array
for emotion in emotions:
    for i in range(1,8):
        complete =url.format(emotion, i)
        quotes = get_quotes(complete)
        X.extend(quotes)
        y.extend([emotion] * len(quotes))
        print(f'Processed page {i} for {emotion}')

Processed page 1 for love
Processed page 2 for love
Processed page 3 for love
Processed page 4 for love
Processed page 5 for love
Processed page 6 for love
Processed page 7 for love
Processed page 1 for religion
Processed page 2 for religion
Processed page 3 for religion
Processed page 4 for religion
Processed page 5 for religion
Processed page 6 for religion
Processed page 7 for religion
Processed page 1 for philosophy
Processed page 2 for philosophy
Processed page 3 for philosophy
Processed page 4 for philosophy
Processed page 5 for philosophy
Processed page 6 for philosophy
Processed page 7 for philosophy
Processed page 1 for happiness
Processed page 2 for happiness
Processed page 3 for happiness
Processed page 4 for happiness
Processed page 5 for happiness
Processed page 6 for happiness
Processed page 7 for happiness


In [10]:
y[400]

'religion'

In [11]:
len(X)

840

In [12]:
len(y)

840

In [13]:
#  preparing dataframe out of scraped data
import pandas as pd

In [14]:
df = pd.DataFrame(list(zip(y, X)), columns=['emotion', 'quotes'])

In [15]:
df.to_csv('emotion.csv')

In [16]:
# making dictionary
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vect = CountVectorizer(max_features=450)

In [18]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [19]:
tokenizer = RegexpTokenizer('\w+')
sw = set(stopwords.words('english'))
ps = PorterStemmer()

In [20]:
def getStemmedQuote(quote):
    quote = quote.lower()
    
    # tokenize
    tokens = tokenizer.tokenize(quote)
    
    # remove stopwords
    new_tokens = [token for token in tokens if token not in sw]
    
    stemmed_token = [ps.stem(token) for token in new_tokens]
    
    clean_quote = ' '.join(stemmed_token)
    
    return clean_quote

def getStemmedQuotes(quotes):
    d = []
    for quote in quotes:
        d.append(getStemmedQuote(quote))
    return d

In [21]:
x = getStemmedQuotes(X)

In [22]:
vect.fit(x)

CountVectorizer(max_features=450)

In [23]:
len(vect.vocabulary_)

450

In [24]:
x_mod = vect.transform(x).todense() # returns matrix form 

In [25]:
x_mod[2]

matrix([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
x_train, x_test, y_train, y_test = train_test_split(
...     x_mod, y, test_size=0.33, random_state=42)

In [28]:
from sklearn.naive_bayes import BernoulliNB
# naive bayes model

In [29]:
model = BernoulliNB()

In [30]:
model.fit(x_train, y_train)

BernoulliNB()

In [31]:
model.score(x_test, y_test)

0.6151079136690647

In [32]:
# To predict


In [33]:
line = "You're just too good to be true can't take my eyes off you you'd be like heaven to touch I wanna hold you so much I love you baby"

In [34]:
X_vec = vect.transform([line]).todense()

In [35]:
model.predict(X_vec)

array(['love'], dtype='<U10')